Projekt Streaming 

In [2]:
import numpy as np
import pandas as pd

np.random.seed(42)

N = 800

df = pd.DataFrame({
	"user_id": range(1, N+1),
	"age": np.random.randint(18, 70, size = N),
	"country": np.random.choice(
		["PL", "DE", "UK", "FR", "ES"], size = N, p=[0.35, 0.2, 0.15, 0.15, 0.15]
	),
	"subscription_type": np.random.choice(
		["basic", "standard", "premium"], size = N, p=[0.45, 0.35, 0.2]
	),
	"monthly_fee": np.random.choice([29, 39, 59], size = N),
	"months_active": np.random.randint(1, 48, size = N),
	"hours_watched_per_month": np.random.normal(45, 18, size = N).clip(5, 140),
	"churned": np.random.choice([0, 1], size = N, p=[0.78, 0.22])
})

df

,user_id,age,country,subscription_type,monthly_fee,months_active,hours_watched_per_month,churned
0,1,56,PL,standard,59,19,55.105237,0
1,2,69,ES,standard,29,12,55.915704,0
2,3,46,UK,standard,59,37,20.049157,0
3,4,32,PL,standard,29,42,71.082544,1
4,5,60,DE,basic,39,17,28.788352,0
...,...,...,...,...,...,...,...,...
795,796,68,UK,basic,29,24,64.029474,0
796,797,22,FR,premium,39,25,27.247959,0
797,798,64,UK,basic,29,45,70.534020,0
798,799,42,DE,standard,29,46,60.170962,1


ZADANIA - ETAP 1 (EDA)

1. Opis Danych

In [3]:
df.head()

,user_id,age,country,subscription_type,monthly_fee,months_active,hours_watched_per_month,churned
0,1,56,PL,standard,59,19,55.105237,0
1,2,69,ES,standard,29,12,55.915704,0
2,3,46,UK,standard,59,37,20.049157,0
3,4,32,PL,standard,29,42,71.082544,1
4,5,60,DE,basic,39,17,28.788352,0


In [ ]:
df.shape

# 800 - wierszy, 8 kolumn

(800, 8)

In [ ]:
df.info() 	# typy danych, brak NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   user_id                  800 non-null    int64  
 1   age                      800 non-null    int32  
 2   country                  800 non-null    object 
 3   subscription_type        800 non-null    object 
 4   monthly_fee              800 non-null    int64  
 5   months_active            800 non-null    int32  
 6   hours_watched_per_month  800 non-null    float64
 7   churned                  800 non-null    int64  
dtypes: float64(1), int32(2), int64(3), object(2)
memory usage: 43.9+ KB


2. Rozkłady:
- subscription_type
- country
- churned w %

In [9]:
# rozkład dla rodzaju subskrypcji

df["subscription_type"].value_counts()

subscription_type
basic       370
standard    298
premium     132
Name: count, dtype: int64

In [10]:
# rozkład dla państw

df["country"].value_counts()

country
PL    255
DE    168
FR    140
ES    120
UK    117
Name: count, dtype: int64

In [32]:
# rozkład dla churned w %

churned_procenty = df["churned"].mean() * 100
print(f"Churn rate: {churned_procenty:.2f}%")

Churn rate: 21.38%


3. Podstawowe pytania biznesowe
- Jaki jest churn rate? 
- Który plan ma najwyższy churn?
- Który kraj ma najwyższy churn?


In [17]:
# Jaki jest churn rate?

churn_rate = df["churned"].mean()

print(f"Churn rate wynosi: {churn_rate:.2f} %")

Churn rate wynosi: 0.21 %


In [33]:
# Który plan ma najwyższy churn ?

# Który plan traci najwięcej użytkowników?

df.groupby("subscription_type")["churned"].mean().sort_values(ascending=False)

subscription_type
premium     0.234848
standard    0.231544
basic       0.191892
Name: churned, dtype: float64

In [31]:
# Który kraj ma najwyższy churn?

df.groupby("country")["churned"].mean().sort_values(ascending=False)

country
UK    0.299145
FR    0.228571
PL    0.211765
DE    0.184524
ES    0.158333
Name: churned, dtype: float64

ZADANIA - ETAP 2 (Feature Engineering)


In [38]:
# 1. Lifetime Value 

# Utworzenie tabeli lifetime_value która zlicza opłate miesięczna i czas aktywny użytkownika w miesiącach
df["lifetime_value"] = df["monthly_fee"] * df["months_active"]

df.head()

,user_id,age,country,subscription_type,monthly_fee,months_active,hours_watched_per_month,churned,lifetime_value
0,1,56,PL,standard,59,19,55.105237,0,1121
1,2,69,ES,standard,29,12,55.915704,0,348
2,3,46,UK,standard,59,37,20.049157,0,2183
3,4,32,PL,standard,29,42,71.082544,1,1218
4,5,60,DE,basic,39,17,28.788352,0,663


In [44]:
# 2. Engagement level

# Na podstawie: hours_watched_per_month:
# "low" -> < 30 godzin
# "medium" -> 30-60 godzin
# "high" -> > 60 godzin

df["engagement_level"] = np.select(
	[
		df["hours_watched_per_month"] < 30,
		df["hours_watched_per_month"].between(30, 60),
		df["hours_watched_per_month"] > 60
	],
	["low", "medium", "high"],
	default = "unknown"
)

df.head(10)

,user_id,age,country,subscription_type,monthly_fee,months_active,hours_watched_per_month,churned,lifetime_value,engagement_level
0,1,56,PL,standard,59,19,55.105237,0,1121,medium
1,2,69,ES,standard,29,12,55.915704,0,348,medium
2,3,46,UK,standard,59,37,20.049157,0,2183,low
3,4,32,PL,standard,29,42,71.082544,1,1218,high
4,5,60,DE,basic,39,17,28.788352,0,663,low
5,6,25,FR,basic,39,37,31.188832,1,1443,medium
6,7,38,DE,basic,39,39,56.482141,0,1521,medium
7,8,56,ES,standard,39,28,39.839019,1,1092,medium
8,9,36,DE,standard,59,41,50.548695,0,2419,medium
9,10,40,FR,standard,29,35,6.181187,0,1015,low


In [ ]:
# 3. Binarne flagi 

'''
tworzymy flagi dla: 

- is_premium -> 1 jeśli premium, else 0
- high_engagement -> 1 jeśli engagement = high
'''
df["is_premium"] = np.where(df["subscription_type"] == "premium", 1, 0)

df.head(5)


,user_id,age,country,subscription_type,monthly_fee,months_active,hours_watched_per_month,churned,lifetime_value,engagement_level,is_premium
0,1,56,PL,standard,59,19,55.105237,0,1121,medium,0
1,2,69,ES,standard,29,12,55.915704,0,348,medium,0
2,3,46,UK,standard,59,37,20.049157,0,2183,low,0
3,4,32,PL,standard,29,42,71.082544,1,1218,high,0
4,5,60,DE,basic,39,17,28.788352,0,663,low,0
5,6,25,FR,basic,39,37,31.188832,1,1443,medium,0
6,7,38,DE,basic,39,39,56.482141,0,1521,medium,0
7,8,56,ES,standard,39,28,39.839019,1,1092,medium,0
8,9,36,DE,standard,59,41,50.548695,0,2419,medium,0
9,10,40,FR,standard,29,35,6.181187,0,1015,low,0


In [49]:
'''
tworzymy flagi dla: 

- high_engagement -> 1 jeśli engagement = high
'''
df["high_engagement"] = np.where(df["engagement_level"] == "high", 1, 0)

df

,user_id,age,country,subscription_type,monthly_fee,months_active,hours_watched_per_month,churned,lifetime_value,engagement_level,is_premium,high_engagement
0,1,56,PL,standard,59,19,55.105237,0,1121,medium,0,0
1,2,69,ES,standard,29,12,55.915704,0,348,medium,0,0
2,3,46,UK,standard,59,37,20.049157,0,2183,low,0,0
3,4,32,PL,standard,29,42,71.082544,1,1218,high,0,1
4,5,60,DE,basic,39,17,28.788352,0,663,low,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,68,UK,basic,29,24,64.029474,0,696,high,0,1
796,797,22,FR,premium,39,25,27.247959,0,975,low,1,0
797,798,64,UK,basic,29,45,70.534020,0,1305,high,0,1
798,799,42,DE,standard,29,46,60.170962,1,1334,high,0,1


In [ ]:
# 4. Normalizacja "na oko"

# dodamy nową kolumnę

df["months_active_ratio"] = df["months_active"] / df["months_active"].max()

df.head()


,user_id,age,country,subscription_type,monthly_fee,months_active,hours_watched_per_month,churned,lifetime_value,engagement_level,is_premium,high_engagement,months_active_ratio
0,1,56,PL,standard,59,19,55.105237,0,1121,medium,0,0,0.404255
1,2,69,ES,standard,29,12,55.915704,0,348,medium,0,0,0.255319
2,3,46,UK,standard,59,37,20.049157,0,2183,low,0,0,0.787234
3,4,32,PL,standard,29,42,71.082544,1,1218,high,0,1,0.893617
4,5,60,DE,basic,39,17,28.788352,0,663,low,0,0,0.361702


In [52]:
# 5. Szybka kontrola

df[["lifetime_value", "months_active", "hours_watched_per_month"]].describe()

,lifetime_value,months_active,hours_watched_per_month
count,800.000000,800.000000,800.000000
mean,1018.328750,24.108750,45.287605
std,678.687046,13.703345,17.700216
min,29.000000,1.000000,5.000000
25%,487.750000,12.000000,33.054976
50%,940.000000,24.000000,45.660241
75%,1404.000000,36.000000,56.920610
max,2773.000000,47.000000,101.194144
